# Exploring the Danceability of a song on Spotify based on multiple characteristics of the song

In [3]:
# Setting up prerequisites
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)